In [1]:
import json
import pandas as pd
import numpy as np
from copy import deepcopy
from IPython.display import display
import re

from modules.team import Team, BenchTeam
from modules.player import Player, Position
from modules.transfer import Transfer
from modules.fixture_difficulty_matrix import FixtureDifficultyMatrix
from modules.utils import getDataFilesSorted

import config

In [2]:
CURRENT_DATE = config.CURRENT_DATE
teams_filename = f"./results/{CURRENT_DATE}/results_{CURRENT_DATE}.json"
scores_filename = f"./results/{CURRENT_DATE}/scores_{CURRENT_DATE}.json"
SELECTED_MODEL_INDEX = 2
TOTAL_BUDGET = 1000

In [3]:
current_team_names = {"Jordan Pickford",
                      "André Onana",
                      "Nathan Collins",
                      "Riccardo Calafiori",
                      "Rayan Aït-Nouri",
                      "Fabian Schär",
                      "Daniel Ballard",
                      "Morgan Rogers",
                      "Bryan Mbeumo",
                      "Alex Iwobi",
                      "Jacob Murphy",
                      "Brennan Johnson",
                      "Hugo Ekitiké",
                      "João Pedro Junqueira de Jesus",
                      "Richarlison de Andrade",
                      }

In [4]:
# TODO: Change how files are loaded

with open(teams_filename, "r") as f:
    tempJson: list[dict] = json.load(f)

actualJson = tempJson[SELECTED_MODEL_INDEX]

In [5]:
all_player_data = pd.DataFrame.from_records(actualJson["players"])

In [6]:
print(all_player_data)

     cost  form  gameweek  ict_index   id                          name  \
0      55   4.7         4       10.5    1             David Raya Martín   
1      44   0.0         4        0.0    2    Kepa Arrizabalaga Revuelta   
2      40   0.0         4        0.0    3                     Karl Hein   
3      40   0.0         4        0.0    4                 Tommy Setford   
4      61   5.7         4       10.7    5  Gabriel dos Santos Magalhães   
..    ...   ...       ...        ...  ...                           ...   
735    55   6.0         4        1.4  736          Gianluigi Donnarumma   
736    45   0.0         4        0.0  737              Jonah Kusi-Asare   
737    45   0.0         4        0.0  738              Lukasz Fabianski   
738    45   0.0         4        0.0  739                 Divine Mukasa   
739    40   0.0         4        0.0  740                 Stephen Mfuni   

    opposing_team  points_per_game  points_this_week position  season  \
0             NFO         

In [7]:
currentTeamPlayers = all_player_data.loc[all_player_data["name"].isin(current_team_names)]


for player in current_team_names:
    if(player not in currentTeamPlayers["name"].values):
        raise ValueError(f"player '{player}' not found")

In [8]:
current_team = Team.fromDataFrame(currentTeamPlayers)

In [9]:
current_team_cost = current_team.getTotalCost()
current_team_cost

954

In [10]:
with open(teams_filename,"r",encoding="utf-8") as f:
    all_data = json.load(f)

all_data = all_data[SELECTED_MODEL_INDEX]
selected_team_df = pd.DataFrame.from_records(all_data["team"])

In [11]:
selected_team = Team.fromDataFrame(selected_team_df)

In [12]:
display(selected_team)

In [13]:
new_players = selected_team - current_team
print(new_players)


Total Score: 131.74468933499531

Goalkeepers:
- David Raya Martín	Score: 11.37	Cost: 55	Fixture Difficulty: 0.0
- Guglielmo Vicario	Score: 9.08	Cost: 51	Fixture Difficulty: 0.0
Defenders:
- Josh Acheampong	Score: 9.9	Cost: 40	Fixture Difficulty: 0.0
- Rico Lewis	Score: 10.35	Cost: 50	Fixture Difficulty: 0.0
- Tyrick Mitchell	Score: 9.37	Cost: 50	Fixture Difficulty: 0.0
Attackers:
- Chris Wood (Vice Captain) 	Score: 13.58	Cost: 76	Fixture Difficulty: 0.0
- Erling Haaland	Score: 13.43	Cost: 141	Fixture Difficulty: 0.0
Midfielders:
- Tijjani Reijnders	Score: 10.48	Cost: 57	Fixture Difficulty: 0.0
- Matt O'Riley	Score: 10.55	Cost: 55	Fixture Difficulty: 0.0
- Mohammed Kudus	Score: 11.12	Cost: 66	Fixture Difficulty: 0.0
- Antoine Semenyo (Captain) 	Score: 14.17	Cost: 75	Fixture Difficulty: 0.0
- Anton Stach	Score: 8.35	Cost: 50	Fixture Difficulty: 0.0


In [14]:
new_team_cost = selected_team.getTotalCost()
new_team_cost

added_cost = new_team_cost - current_team_cost
print("Added cost:",added_cost)

surplus = current_team_cost - new_team_cost


Added cost: -18


In [15]:
#HEURISTIC = "combined"
#MODE = SolverMode.CHEAPEST_FIRST

In [16]:
# all_player_data["score"] = all_player_data[HEURISTIC] * all_player_data["form"]
deviation = np.std(all_player_data["score"])
scale_factor = deviation
scale_factor, deviation

(2.5342070487231494, 2.5342070487231494)

In [17]:
START_SAMPLE_GAMEWEEK = config.CURRENT_GAMEWEEK
END_SAMPLE_GAMEWEEK = min(config.CURRENT_GAMEWEEK+2, 39)

In [18]:
matrix = FixtureDifficultyMatrix(scale_factor, START_SAMPLE_GAMEWEEK, END_SAMPLE_GAMEWEEK, config.CURRENT_SEASON)
matrix.precomputeFixtureDifficulty(scale_factor)

new_players.recalculateFixtureDifficulty(matrix)
# new_players.calculateScore(HEURISTIC)

# selected_team.recalculateFixtureDifficulty(matrix)
# selected_team.calculateScore(HEURISTIC)

# current_team.recalculateFixtureDifficulty(matrix)
# current_team.calculateScore(HEURISTIC)

In [19]:
print("Current Team:")
display(current_team)
print()
print("Selected Team:")
display(selected_team)

Current Team:



Selected Team:


In [20]:
score_dif = selected_team.getTotalScore() - current_team.getTotalScore()
print("Score difference:",score_dif)

Score difference: 85.82398746951984


In [21]:
def getBest(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team, pPosition: Position):
    # TODO: Store Team players in sorted tree instead of list
    currentPositionData = pCurrentTeam.getPlayersListByPosition(pPosition)
    newPlayersPositionData = pNewPlayers.getPlayersListByPosition(pPosition)
    oldTotalCost = pCurrentTeam.getTotalCost()
    maxCost = max(oldTotalCost, TOTAL_BUDGET)
    #maxCost = TOTAL_BUDGET

    allTransfers = []

    for i in range(len(currentPositionData)):
        oldPlayer = currentPositionData[i]
        oldPlayerCost = oldPlayer.getCost()
        for j in range(len(newPlayersPositionData)):

            newPlayer = newPlayersPositionData[j]
            newCost = oldTotalCost - oldPlayerCost + newPlayer.getCost()

            if (newCost <= maxCost):
                allTransfers.append(Transfer(oldPlayer, newPlayer))

    if (len(allTransfers) >= 1):
        bestTransfer = max(allTransfers)
        if(bestTransfer.getScoreDif() > 0):
            return bestTransfer
    else:
        return None

In [22]:
def get_updated_team(team: pd.DataFrame, old_player: pd.Series, new_player: pd.Series):
    team: pd.DataFrame = team.drop(index=old_player.name)
    team.loc[len(team)] = new_player
    return team

In [23]:
def get_bench(team: pd.DataFrame):
    positions = ["FWD","DEF","MID","GKP"]
    team = team.reset_index()
    bench = pd.DataFrame(columns=team.columns)
    for position in positions:
        worst_player_index = team.loc[team["position"]==position]["score"].idxmin()
        worst_player = team.loc[worst_player_index].copy()
        bench.loc[len(bench)] = worst_player
        team = team.drop(index=worst_player_index)
    return team, bench

In [24]:
def getBestTransferNew(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team) -> Transfer | None:
    positions = Position.listValues()
    bestTransfers = []
    for position in positions:
        transfer = getBest(pCurrentTeam, pNewTeam, pNewPlayers, position)
        if(transfer is not None):
            bestTransfers.append(transfer)
    actualBestTransfer = max(bestTransfers)
    return actualBestTransfer

In [25]:
def getNewTeam(pCurrentTeam: Team, pSelectedTeam: Team, pNewPlayers: Team, pMatrix: FixtureDifficultyMatrix):

    TRIPLE_CAPT_THRESHOLD = 0.1

    transferData = getBestTransferNew(pCurrentTeam, pSelectedTeam, pNewPlayers)
    transferData.updateFixtureDifficulties(pMatrix)
    print("Best transfer:")
    print(transferData)
    oldPlayer = transferData.getOldPlayer()
    newPlayer = transferData.getNewPlayer()
    if transferData is None:
        return
    
    changingPosition: Position = transferData.getPosition()
    
    newTeam = deepcopy(pCurrentTeam)
    playersOfPosition = newTeam.getPlayersListByPosition(changingPosition)

    for i in range(len(playersOfPosition)):
        player = playersOfPosition[i]
        if (player.getId() == oldPlayer.getId()):
            newTeam.removePlayerByIndex(i, changingPosition)
            newTeam.addPlayer(newPlayer)
    

    newTeam.recalculateFixtureDifficulty(pMatrix)

    for player in newTeam.getPlayers():
        if(player.getCurrentDifficulty() < TRIPLE_CAPT_THRESHOLD):
            print(f"Suggested triple captain: {player.getName()}")
    return newTeam

In [26]:
newTeam = getNewTeam(current_team, selected_team, new_players, matrix)
newTeam = newTeam.toBenchTeam()
display(newTeam)
display(f"New total score: {newTeam.getTotalScore()}")

Best transfer:
Transfer from Jacob Murphy -> Antoine Semenyo:
Old player: Jacob Murphy	Score: 1.14	Cost: 63	Fixture Difficulty: 0.46827738943797304
New player: Antoine Semenyo (Captain) 	Score: 14.17	Cost: 75	Fixture Difficulty: 0.6335517621807876
- Cost change: 12
- Score change: 13.03


'New total score: 101.30933863030042'